<a href="https://colab.research.google.com/github/a07458666/TBrainNLP/blob/master/1_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from google.colab import drive
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')
os.chdir(r"/content/drive/My Drive/tbrainData/")
# 讀入官方的data
newsDf = pd.read_csv('train_final.csv')
newsDf.head()

## 文章前處理
刪除 英文 數字 特殊符號

In [ ]:
newsDf["article"] = newsDf["article"].apply(lambda x: re.sub("[-／〔〕【】（） 「」()：:0-9A-Za-z\n\r//、.%，▲◆▪<>#*○]", "", str(x)))
newsDf["article"] = newsDf["article"].apply(lambda x: re.sub("[!！？?]", "。", str(x)))

newsDf.head(50)

## 將文章分成句子
因為文章太長 不可能一次進入模型
所以一個句子一個句子預測名字

In [ ]:
def split(word):
    return [char for char in word]

def BIO_encode(content, names):
    if str(names) == "['']": 
      Context_list, BIO_list = split(content)+[''], np.repeat('O', len(content)+1).tolist()
    else:

      start_idx = []
      end_idx = []
      name_list = []
      NER_list = []

      for name in names:
          # name = truth[0]
          name_start = [m.start() for m in re.finditer(name, content)]
          name_end = list(np.asarray(name_start) + len(name))

          # 紀錄entity & 名字名稱
          NER_list += np.repeat('PERSON', len(name_start)).tolist()
          name_list += np.repeat(name, len(name_start)).tolist()
          
          # 名字在文章中的start, end
          start_idx += name_start
          end_idx += name_end

      # 列出剛剛找到的entity在文章中的資訊
      print('zip ', start_idx, end_idx, NER_list, name_list)
      entity_info_list = [set(zip(start_idx, end_idx, NER_list, name_list))]
      if str(entity_info_list) == '[set()]':
        Context_list, BIO_list = split(content)+[''], np.repeat('O', len(content)+1).tolist()
        return Context_list, BIO_list

      print('entity_info_list ', entity_info_list)

      review_PER = []
      for entity in sorted(entity_info_list[0]):
          # print(entity)
          if entity[2] == 'PERSON':
              review_PER.append(entity[3])

      Context_list=[]
      BIO_list = []
      sorted_entity_info_list = sorted(entity_info_list[0])

      for i, entity_info in enumerate(sorted_entity_info_list):
          word_start_loc, word_end_loc, flag, word = entity_info
          # 補O
          if word_start_loc != 0:
              if i != 0:
                  O_start_loc = sorted(sorted_entity_info_list)[i-1][1]
              else:
                  O_start_loc = 0

              O_end_loc = word_start_loc
              add_O_string = content[O_start_loc:O_end_loc]
              sub_split_words = split(add_O_string)
              print(sub_split_words)
              sub_BIO = np.repeat('O', len(add_O_string)).tolist()

              Context_list += sub_split_words
              BIO_list += sub_BIO
          
          # 當圈人名
          sub_split_words = split(word)

          # 人名
          if (flag == 'PERSON') or (flag == 'ORG') or (flag == 'GPE'):

              # 人名
              if (flag == 'PERSON'):
                  flag2 = 'PER'
              # 機構名
              if (flag == 'ORG'):
                  flag2 = 'ORG'
              # 地名
              if (flag == 'GPE'):
                  flag2 = 'LOC'

              if len(word) > 1:
                  sub_BIO = ['B-' + flag2] + np.repeat('I-' + flag2, len(word) - 1).tolist()
              else:
                  sub_BIO = ['B-' + flag2]
          else:
              flag2 = 'O'
              sub_BIO = np.repeat(flag2, len(word)).tolist()
          
          Context_list += sub_split_words
          BIO_list += sub_BIO

          # 上面處理完，但最後一個人名後面還有一堆字需要補O
          if word_end_loc != len(content) and i == (len(sorted_entity_info_list)-1):
              word = content[word_end_loc:]
              sub_split_words = split(word)
              sub_BIO = np.repeat('O', len(word)).tolist()

              Context_list += sub_split_words
              BIO_list += sub_BIO

      Context_list.append('')
      BIO_list.append('')
    return Context_list, BIO_list

In [ ]:
all_content = []
all_BIO = []
all_article_index = []
for index, news_info in newsDf.iterrows():
  '''
  if index > 37:
    break
  if index < 37:
    continue
  '''
  input_data = str(news_info['article'])
  names = news_info['name'].replace("'", "").replace("[", "").replace("]", "").split(', ')
  if input_data != '' and len(input_data) > 2:
    input_datas = input_data.split('。')
    for input in input_datas:
      if input != '':
        print('index ', index,' names ', names,'\n input_datas ', input)
        context_list, BIO_list = BIO_encode(content = input, names = names)
        all_content.append(context_list)
        all_BIO.append(BIO_list)
        all_article_index.append(index)
        #print('index ', index,' content_lists ', context_list,'\n BIO_lists ', BIO_list)
        #for i in range(len(context_list)):
        #  print('i = ', i, '  ', context_list[i], ' = ', BIO_list[i])
        #print('len context_list= ', len(context_list), 'len BIO_list = ', len(BIO_list))

print(all_content)
print(all_BIO)

In [ ]:
np.save('all_content', all_content)
np.save('all_BIO', all_BIO)
np.save('all_article_index', all_article_index)